# Finding base station neighbours via the Delaunay triangulation

We will reuse the work of Delphine PAQUIRY to find base stations' neighbours.

In [ ]:
# Importation of libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay


In [ ]:
# Importing database
df = pd.read_csv("../database/data.csv", sep=";", decimal=",")
df.head()

## Useful methods

In [ ]:
def plot_delaunay(points, delaunay_triangulation):
    """ Plots a Delaunay triangulation
        IN:
            points: Coordinates of points to triangulate (ndarray)
            delaunay_triangulation: Result of the Delaunay triangulation
        OUT: 
            plot
    """
    plt.triplot(points[:,0], points[:,1], delaunay_triangulation.simplices, linewidth=0.3)
    plt.plot(points[:,0], points[:,1], 'o', markersize=3, c="green")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.show()

## First try on the Gard county

### Creation of the dataframe

In [ ]:
department = "Gard"

In [ ]:
df_gard = df.loc[df['nom_dep'] == department]
df_gard = df_gard.drop(columns=["code_op", "id_site_partage", "id_station_anfr", "nom_reg", "nom_dep", "insee_dep", "insee_com", "site_2g", "site_3g", "site_4g", "site_5g", "mes_4g_trim", "site_ZB", "site_DCC", "site_strategique", "site_capa_240mbps", "date_ouverturecommerciale_5g", "site_5g_700_m_hz", "site_5g_800_m_hz", "site_5g_1800_m_hz", "site_5g_2100_m_hz", "site_5g_3500_m_hz"])
df_gard.head()

### Base stations' number

In [ ]:
providers = list(np.unique(df_gard["nom_op"]))
nb_pro = len(providers)

df_proPerDep = pd.DataFrame({
    "nom_dep" : department,
    "Free Mobile"   : [None],
    "SFR"   : [None],
    "Orange"   : [None],
    "Bouygues Telecom"   : [None],
    "Total"   : [None]
})

for pro in providers: # number of sites per provider
    count = list(df_gard["nom_op"]).count(pro)
    df_proPerDep.loc[df_proPerDep["nom_dep"]==department, pro] = count
df_proPerDep.loc[df_proPerDep["nom_dep"]==department, "Total"] = len(df_gard["num_site"])

In [ ]:
df_proPerDep

### Delaunay triangulation on Free Mobile stations

In [ ]:
# Selecting the provider : Free Mobile
df_gard_free = df_gard.loc[df['nom_op'] == "Free Mobile"]
df_gard_free = df_gard_free.drop(columns=['nom_op'])
df_gard_free.head()

In [ ]:
# Creation of points couples for Delaunay
df_gard_free_points = np.array(df_gard_free[['longitude', 'latitude']])

In [ ]:
delaunay_gard_free = Delaunay(df_gard_free_points)

In [ ]:
plot_delaunay(df_gard_free_points, delaunay_gard_free)

### Addition of a distance condition

In [ ]:
delaunay_gard_free.simplices

In [ ]:
import networkx as nx
from itertools import combinations

In [ ]:
tmp = combinations(delaunay_gard_free.simplices[1], 2)
tmp

In [ ]:
G = nx.Graph()
G.add_nodes_from(range(len(df_gard_free)))

for simplex in delaunay_gard_free.simplices:
    G.add_edges_from(combinations(simplex, 2))
# for i, node in enumerate(G.nodes()):
#     G.nodes[node]['num_site'] = df_gard_free.iloc[i]['num_site']    
# nx.set_node_attributes(G, pos, 'pos')

nx.draw(G)

## Big Delaunay on France
Only Free Mobile

In [ ]:
# Selecting the provider : Free Mobile
df_free = df.loc[df['nom_op'] == "Free Mobile"]
df_free = df_free.drop(columns=['nom_op'])
df_free.head()

In [ ]:
# Creation of points couples for Delaunay
df_free_points = np.array(df_free[['longitude', 'latitude']])

In [ ]:
delaunay_free = Delaunay(df_free_points)

In [ ]:
plot_delaunay(df_free_points, delaunay_free)